# DeFi Anti-Fraud

### FCS, 2023

#### Выполнили: Денисов Степан, Карлов Владимир 

***Description:***   

В данном проекте мы попытаемся классифицировать вредоносные смарт-контракты используя его байткод.
Стоить заметить, что задача будет предсказать именно метки 1 и 0, а не просто получить скоры :)

Проект будет состоять из двух частей:
1) ML часть 
2) Инфраструктурная часть

Давайте начнем с ML части и разберемся с моделью.

In [ ]:
import pandas as pd
from collections import Counter
import torch
import numpy as np
import random

def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


SEED = 42
set_random_seed(SEED)

In [2]:
data = pd.read_parquet('Data/malicious_contract_training_dataset_final.parquet', engine='fastparquet')

In [4]:
data.head(3)

,contract_address,contract_name,contract_etherscan_label,contract_tag,contract_creator,contract_creation_tx,creation_bytecode,decompiled_opcodes,malicious,contract_creator_tag,source,notes,contract_creator_etherscan_label,block
82965,0x04ae3226c80e8c04d35e6e56089345bdd06da6de,,,Multichain Exploiter 10,0x98f5b0a99f258a09403d965647ee9d53677914af,0xe962b06db95db1dedb8b1664bf93bb8816c0cfbd1876...,0x6080604052600180546001600160a01b03191673c02a...,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 DUP1 S...,True,,Luabase ethereum.tags table,,exploit,14041572
82966,0xc5ac25cfc2b8284e84ca47dad21cf1319f732c11,,,,0x07e02088d68229300ae503395c6536f09179dc3e,0xae03b08f220e5d7750e97e67ea85ac6e69649f6fd6d8...,0x60a0604052600180553373ffffffffffffffffffffff...,PUSH1 0xa0 PUSH1 0x40 MSTORE PUSH1 0x01 DUP1 S...,True,xToken Exploiter,Luabase ethereum.tags table,,heist,12419856
82967,0x79dbe9bbde91a35fa8148a14084979a531fe57ea,,,,0x11112f684cb88d43ca0e132e585e882606063fbe,0x07766fa0cb09dc5221763049c4a2fbbf3556c8660490...,0x6080604052600080546001600160a01b031990811673...,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x00 DUP1 S...,True,Malicious Actor (Exploit Abuser),Luabase ethereum.tags table,,heist,13693565


In [5]:
data.shape

(50927, 14)

In [6]:
data.decompiled_opcodes.iloc[0]

'PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 DUP1 SLOAD PUSH1 0x01 PUSH1 0x01 PUSH1 0xa0 SHL SUB NOT AND PUSH20 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 OR SWAP1 SSTORE CALLVALUE DUP1 ISZERO PUSH2 0x0036 JUMPI PUSH1 0x00 DUP1 REVERT JUMPDEST POP PUSH2 0x0040 CALLER PUSH2 0x0045 JUMP JUMPDEST PUSH2 0x0095 JUMP JUMPDEST PUSH1 0x00 DUP1 SLOAD PUSH1 0x01 PUSH1 0x01 PUSH1 0xa0 SHL SUB DUP4 DUP2 AND PUSH1 0x01 PUSH1 0x01 PUSH1 0xa0 SHL SUB NOT DUP4 AND DUP2 OR DUP5 SSTORE PUSH1 0x40 MLOAD SWAP2 SWAP1 SWAP3 AND SWAP3 DUP4 SWAP2 PUSH32 0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0 SWAP2 SWAP1 LOG3 POP POP JUMP JUMPDEST PUSH2 0x0605 DUP1 PUSH2 0x00a4 PUSH1 0x00 CODECOPY PUSH1 0x00 RETURN UNKNOWN_0xfe PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x04 CALLDATASIZE LT PUSH2 0x008a JUMPI PUSH1 0x00 CALLDATALOAD PUSH1 0xe0 SHR DUP1 PUSH4 0x9dc29fac GT PUSH2 0x0059 JUMPI DUP1 PUSH4 0x9dc29fac EQ PUSH2 0x0128 JUMPI DUP1 PUSH4 0xbebbf4d0 EQ PUSH2 0x015b JUMPI DUP1 PUSH4 0xd505accf EQ PUSH2 0x0

In [60]:
data = data[["decompiled_opcodes", "malicious"]]
data["malicious"] = data["malicious"].astype(int)

/var/folders/cv/6cyws3yn2dv5hh29wrf32vv80000gn/T/ipykernel_45330/35760649.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["malicious"] = data["malicious"].astype(int)


Ага, что мы имеем? 

Есть колонка 'decompiled_opcodes' - это и будет нашими фичами

Malicious - наш таргет

In [64]:
data.malicious.value_counts() / len(data)

malicious
0    0.997506
1    0.002494
Name: count, dtype: float64

Как видно, у нас имеется значительный дисбаланс классов, что на самом деле и соответствует реальности. 

Как ни крути, не все юзеры блокчейна занимаются непотребствами.

## Предобработка данных (1 балл)

Давайте предобработаем данные, важно понимать что в байткоде будут иметься сигнатуры и адреса других контрактов

Может ли это быть полезно?

Это будет зависеть от вас

В данной секции от вас ожидается предобработка данных

Визуализация и анализ природы данных - необязательны, но желательны, если найдете что-то интересное - мы это вам зачтем в виде бонуса +1 балл :) 

### Train / test split

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
X = data.drop(columns='malicious')
y = data.malicious.astype(int)

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.40, random_state=SEED, stratify=y)
X_test, X_val, y_test, y_val = train_test_split( X_test, y_test, test_size=0.30, random_state=SEED, stratify=y_test)

In [68]:
## Вжух - ваш код тут

## Модель (5 баллов)

Итак перед вами стоит задача классифицировать смарт-контракты.

Представьте что вы глава проекта из https://cyvers.ai/ 

Вы разрабатываете софт для фондов, которые вкладывает деньги в различные DeFi протоколы и им важно своевременно выводить деньги.

Перед вами стоит задача разработать предиктивный софт по DeFi безопасности, возникает несколько вопросов:

1) Выбор модели на ваше усмотрение, помните что потом эту модель вам нужно будет использовать в инфраструктурном коде
* хочется поэксперементировать с разными вариантами:
    * бейзлайн -- выделить признаки каким-либо статистическим способом из текста (здесь мы выбрали самое сложное из простого -- )
2) Помните, что модель нужна для своевременного реагирования на угрозу, важна ли здесь скорость инференса?
3) Является ли более предпочтительной интепретируемость, либо более эффективная модель, но блек бокс?
4) Как вы думаете, что важнее в нашей задаче precision или recall?
5) Как именно вы будете работать с дисбалансом классом, а именно какую метрику выберете?

Все это решать вам)

Ваши рассуждения и комментарии в ноутбуке - также влияют на оценку, поэтому не стесняйтесь рассуждать, важна ваша мысль

### Baseline -- linear model / boosting on TF-IDF embeddings with ngrams

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 1))

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = vectorizer.fit_transform(data.decompiled_opcodes)
vectorizer.get_feature_names_out()
print(X.shape)

(50927, 526346)


In [77]:
X_train_features = vectorizer.transform(X_train.decompiled_opcodes)

In [79]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=SEED,).fit(X_train_features, y_train)


In [80]:
X_test_features = vectorizer.transform(X_test.decompiled_opcodes)

In [81]:
y_pred_test = clf.predict(X_test_features)

In [84]:
y_pred_test.sum()

0

In [85]:
from sklearn.metr

36

## Анализ результатов и ваши метрики (1 балл)

### Посмотрим на метрики

In [ ]:
y_pred = # ваши скоры на тесте
y_pred_c = # метки класса на тесте
y_test = # метки на тесте

print('------------------------------------TESTING INFO-------------------------------------------------')
# Метрики которые вы считаете важными
print(f'F1 on test - {f1_score(y_test, y_pred_c)}')
print(f'precision on test - {precision_score(y_test, y_pred_c)}')
print(f'recall on test - {recall_score(y_test, y_pred_c)}')

cm = confusion_matrix(y_test, y_pred_c, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[0,1])
disp.plot()
plt.show()

### Посмотрим на распределение скоров модели на тесте

In [ ]:
plt.hist(y_pred[y_test==0])
plt.show()
plt.hist(y_pred[y_test==1])
plt.show()

### Интерпретируемость модели

In [25]:
# Вжух ваша интерпретация если она есть как вы думаете логичны ли выводы?

### Тесты на реальных данных

Test for https://etherscan.io/address/0xcff07c4e6aa9e2fec04daaf5f41d1b10f3adadf4

This is a https://rekt.news/teamfinance-rekt/ malicious contract

In [ ]:
from test_data import team_finance_exploit

In [26]:
# Вжух посмотрим на результаты?

## Сохранение модели

In [28]:
# Сохраняем в директорию Model/ и идем в инфраструктурную часть 